### Initial Tests with `python-docx`

The following is a serie of tests performed with the library `python-docx` in order to standarize the layout of the TrisoNIM document.

In [118]:
# Import required libraries

from docx import Document
from docx.shared import Cm, Inches, Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT, WD_COLOR_INDEX
from docx.enum.table import WD_ROW_HEIGHT_RULE
from docx.oxml.ns import qn
from docx.oxml import OxmlElement


In [122]:
document = Document()

# Page Layout
section = document.sections[0]

# Set custom margins
section.top_margin = Cm(2)
section.bottom_margin = Cm(2.5)
section.left_margin = Cm(2)
section.right_margin = Cm(1.84)

# Header Settings
section = document.sections[0]
header = section.header

# Add picture to header
header_paragraph = header.paragraphs[0] if header.paragraphs else header.add_paragraph()
run = header_paragraph.add_run()
run.add_picture("NIM_Logo-horizontal_color_Header.png", width = Cm(5))
header_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.LEFT

# Insert "NIM" Logo
document.add_picture("NIM_triso_color_AF.png", width = Cm(5.3))

# Setting the default font style for the document
style = document.styles["Normal"]
style.font.name = "Ubuntu"
style.font.size = Pt(8)
style.paragraph_format.line_spacing = 1.0

# Add title and set formatting
title = document.add_paragraph()
run = title.add_run("Limitaciones del test:")
run.font.name = "Ubuntu"
run.font.size = Pt(10)
run.bold = True
title.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

# "Limiting factors" intro phrase
intro_phrase_limitations = document.add_paragraph("Algunas condiciones que pueden afectar a la precisión del estudio son:")
for run in intro_phrase_limitations.runs:
    run.font.name = "Ubuntu"
    run.font.size = Pt(8)

# Main bullet points
bullet_points_test_limitations = [
    "Edad gestacional inferior a 10 semanas en el momento del análisis.",
    "Transfusión sanguínea alogénica reciente; transplante o terapia de células madre.",
    "Embarazos de tres o más fetos.",
    ("Los resultados de la prueba pueden no refejar los cambios genéticos del feto en aquellas situaciones donde los cambios genéticos estén limitados a la placenta (mosaicismo confinado a la placenta), cuando estén presentes células genética o  cromosómicamente normales y anormales en el feto (mosaicismo fetal) o en la madre (mosaicismo materno), o en  presencia de alteraciones cromosómicas parciales."),
    "La presencia de una alteración cromosómica en regiones no analizadas o alteraciones cromosómicas en los progenitores.",
    "Triploidías/ tetrasomías completas.",
    "Microdeleciones cromosómicas en las regiones analizadas de un tamaño inferior al límite de resolución de la técnica. Límite establecido en 7Mb, excepto en las modalidades de Excellence y NeoSeq, que es de 5Mb, y de 3MB en el Síndrome deDigeorge 22q11.2.",
    "La obtención de una baja fracción fetal.",
    "En pacientes con obesidad mórbida (BMI≥35) o en tratamiento con heparina de bajo peso molecular, La tasa de no informatividad puede verse incrementada."
]

for text in bullet_points_test_limitations:
    paragraph = document.add_paragraph(text, style = "List Bullet")
    paragraph_format = paragraph.paragraph_format
    paragraph_format.left_indent = Inches(0.295)
    paragraph_format.first_line_indent = Inches(0)
    paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
    paragraph.paragraph_format.space_after = Pt(3)
    paragraph.paragraph_format.space_before = Pt(3)


# "Limiting factors" outro phrase
outro_phrase_limitations = document.add_paragraph("Asimismo, la tecnología utilizada no permite detectar otras alteraciones numéricas no especificadas, grandes deleciones, reordenamientos (como translocaciones, inversiones, o anillos cromosómicos), disomías uniparentales, cambios en la secuencia o fuera de las variantes especificadas en NeoSeq, expansiones repetitivas de trinucleótidos o alteraciones epigenéticas (como alteraciones de los centros de impronta), que podrían causar las mismas o similares patologías a las asociadas a algunas microdeleciones o microduplicaciones incluidas en esta prueba.")
outro_phrase_limitations.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY
for run in outro_phrase_limitations.runs:
    run.font.name = "Ubuntu"
    run.font.size = Pt(8)

# Contents for reference calculations:
reference_calculations_table = (
    ('Trisomía 21', '99.17 %', '99.95 %', '92.19 %', '99.99 %'),
    ('Trisomía 18', '98.24 %', '99.95 %', '76.61 %', '99.99 %'),
    ('Trisomía 13', '99.90 %', '99.96 %', '32.84 %', '99.99 %'),
    ('Aneup. Sex', '85.70 %', '99.90 %', '', ''),
    ('Cromosoma Y', '99.53 %', '', '', '')
)

# Create and fill table of "Reference Calculations":
table = document.add_table(rows = 1, cols = 5)
table.autofit = False
table.allow_autofit = False

hdr_cells = table.rows[0].cells
headers = [" ", "Sensibilidad", "Especifidad", "VPP", "VPN"]
for i, header in enumerate(headers):
    paragraph = hdr_cells[i].paragraphs[0]
    run = paragraph.add_run(header)
    run.font.name = "Ubuntu"
    run.font.size = Pt(7)

# Fill in the table with the data
for data_row in reference_calculations_table:
    row_cells = table.add_row().cells
    for i, item in enumerate(data_row):
        paragraph = row_cells[i].paragraphs[0]
        run = paragraph.add_run(item)
        run.font.name = "Ubuntu"
        run.font.size = Pt(7)

# Formatting table cells
for row in table.rows:
    row.height = Inches(0.1)
    for cell in row.cells:
        cell.width = Inches(0.8)

disclaimer_text = document.add_paragraph()
run = disclaimer_text.add_run("nim")
run.bold = True
run.font.color.rgb = RGBColor(255, 132, 146)

run = disclaimer_text.add_run("genetics S.L participa activamente y de forma periódica en programas de evaluación externa de la calidad de ámbito internacional organizados por la ")
run.bold = False

run = disclaimer_text.add_run("European Molecular Genetics Quality Network ")
run.italic = True

run = disclaimer_text.add_run("(EMQN), ")
run.italic = False

run = disclaimer_text.add_run("Cytogenetics European Quality Association ")
run.italic = True

run = disclaimer_text.add_run("(CEQA), ")
run.italic = False

run = disclaimer_text.add_run("United Kingdom National External Quality Assessment Service ")
run.italic = True

run = disclaimer_text.add_run("(UKNEQAS), y otras organizaciones de evaluación.")
run.italic = False

for run in disclaimer_text.runs:
    run.font.name = "Ubuntu"
    run.font.size = Pt(8)

disclaimer_text.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

# Footer Settings
section = document.sections[0]
footer = section.footer
footer_paragraph = footer.paragraphs[0] if footer.paragraphs else footer.add_paragraph()
run = footer_paragraph.add_run("nim")
run.bold = True
run.italic = True
run = footer_paragraph.add_run("""genetics, genómica y medicina S.L. Centro de Diagnóstico Analítico con Unidad de Genética.""")
run.italic = True
run = footer_paragraph.add_run("""
Nº de licencia CS19067 """)
run.italic = True
run.font.color.rgb = RGBColor(255, 132, 146)
run = footer_paragraph.add_run("""V3.1 01/11/2022""")
run.italic = True
footer_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.JUSTIFY

# Add page number
footer_paragraph = footer.add_paragraph()
run = footer_paragraph.add_run()
fldChar1 = OxmlElement("w:fldChar")
fldChar1.set(qn("w:fldCharType"), "begin")
instrText = OxmlElement("w:instrText")
instrText.set(qn("xml:space"), "preserve")
instrText.text = "PAGE   \\* MERGEFORMAT"
fldChar2 = OxmlElement("w:fldChar")
fldChar2.set(qn("w:fldCharType"), "end")
run._r.append(fldChar1)
run._r.append(instrText)
run._r.append(fldChar2)
footer_paragraph.alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT

# Add page break
document.add_page_break()

# Save document
document.save("table_test.docx")

### Remarks:

- The second page of the document includes information regarding the type of test:
    - TrisoNIM BASIC
        - Is BASIC PLUS or BASIC 22 still in use?
    - TrisoNIM ADVANCE
        - ADVANCE 24 & ADVANCE are the same?
    - TrisoNIM PREMIUM
        - Are all PREMIUM 24DG?
    - TrisoNIM EXCELLENCE
        - 
    - TrisoNIM NEOSEQ
        - 